#  Overview

This notebook provides step-by-step details on the calculations for Criterion B.

# Setup

In [1]:
import os

import ee
from gee_redlist.ee_rle import make_eoo, area_km2, get_aoo_grid_projection
import geopandas as gpd
import ipywidgets as widgets
from lonboard import Map, PolygonLayer, BitmapTileLayer

from gee_redlist.ee_auth import initialize_ee
from gee_redlist.ee_rle import load_yaml

In [2]:
# Initialize Earth Engine
initialize_ee(project=os.environ['GOOGLE_CLOUD_PROJECT'])

# Analysis

In [3]:
# Define the ecosystem that this notebook is analyzing
ecosystem_code = 'MMR-T1.1.1'

In [4]:
# Load the country config
country_config_path = os.environ['VSCODE_CWD'] + '/config/country_config.yaml'
country_config = load_yaml(country_config_path)
print(f'{country_config = }')

gee_project_path = country_config['gee_project_path']

country_config = {'country_name': 'Myanmar', 'country_code': 'MM', 'gee_project_path': 'projects/goog-rle-assessments/assets', 'classified_image': {'asset_id': 'mm_ecosys_v7b', 'classes': [{'id': 52, 'name': 'Tanintharyi island rainforest', 'code': 'MMR-T1.1.1'}, {'id': 54, 'name': 'Tanintharyi Sundaic lowland evergreen rainforest', 'code': 'MMR-T1.1.2'}, {'id': 53, 'name': 'Tanintharyi limestone tropical evergreen forest', 'code': 'MMR-T1.1.3'}]}}


In [5]:
# Extract the class info for the ecosystem
class_info = [x for x in country_config['classified_image']['classes'] if x['code'] == ecosystem_code][0]
print(f'{class_info = }')

ecosystem_image = {
    'asset_id': f"{gee_project_path}/{ecosystem_code}/{class_info['id']}",
    'pixel_value': class_info['id']
}
print(f'{ecosystem_image = }')

class_info = {'id': 52, 'name': 'Tanintharyi island rainforest', 'code': 'MMR-T1.1.1'}
ecosystem_image = {'asset_id': 'projects/goog-rle-assessments/assets/MMR-T1.1.1/52', 'pixel_value': 52}


In [6]:
classified_image_asset_id = f"{country_config['gee_project_path']}/{country_config['classified_image']['asset_id']}"
print(f'{classified_image_asset_id = }')

class_img = (
    ee.Image(classified_image_asset_id)
      .eq(ecosystem_image['pixel_value'])
      .selfMask()
)
print(f'class_img: {class_img.getInfo()}')

classified_image_asset_id = 'projects/goog-rle-assessments/assets/mm_ecosys_v7b'
class_img: {'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [14299, 23576], 'crs': 'EPSG:4326', 'crs_transform': [0.0008084837557075694, 0, 89.60991403135986, 0, -0.0008084837557075694, 28.548369897789982]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[94.66768735253133, 28.548775583027798], [93.22272474215634, 28.548775580809586], [91.41652148658464, 28.548775539452144], [89.60939625454633, 28.54877430628462], [89.60949715757825, 9.485667642440385], [91.41652148658464, 9.486595817478428], [92.86148408075269, 9.486595814990817], [94.30644671824773, 9.486595784089035], [95.39016867380931, 9.48715124036664], [96.47389058147114, 9.486595770655939], [97.91885317497325, 9.486595836813008], [99.36381579356966, 9.486595799973498], [101.1708401374696, 9.48566765739743], [101.17094098108427, 28.548774348

In [7]:
# Commented out until bug is fixed: https://github.com/developmentseed/lonboard/issues/1064
# # Determine the coordinates for viewing the image
# longitude, latitude = class_img.geometry().centroid().getInfo()['coordinates']
# longitude, latitude

# Extent of occurrence (EOO) (subcriterion B1)

## Detailed steps

Set the scale (in meters) for reducing the image pixels to polygons. Use the image's nominal scale unless is is less than 50 meters per pixel.

In [8]:
reduction_scale = max(class_img.projection().nominalScale().getInfo(), 50)
reduction_scale

90

In [9]:
gridcells = class_img.updateMask(1).reduceToVectors(
    scale=reduction_scale,
    geometry=class_img.geometry(),
    geometryType='polygon',
    maxPixels=1e12,
    bestEffort=False
)

# convexHull() is called twice as a workaround for a bug
# (https://issuetracker.google.com/issues/465490917)
hull = gridcells.geometry().convexHull(maxError=1).convexHull(maxError=1)

In [10]:
gridcells_gdf = gpd.GeoDataFrame.from_features(
    gridcells.getInfo()['features'],
    crs='EPSG:4326'
)
print(f'gridcells_gdf: {gridcells_gdf.shape}')

hull_gdf = gpd.GeoDataFrame.from_features(
    ee.FeatureCollection(hull).getInfo(),
    crs='EPSG:4326'
)

gridcells_gdf: (605, 3)


In [11]:
tile_url = class_img.getMapId(
    vis_params={
        'palette': ['red'],
        'opacity': 0.5
    }
)['tile_fetcher'].url_format

In [12]:
gridcells_layer = PolygonLayer.from_geopandas(
    gridcells_gdf,
    get_fill_color=[255, 0, 0, 127],
    stroked=True,
    get_line_width=2,
    get_line_color=[0, 0, 0, 150],
    # auto_highlight=True,
)

hull_layer = PolygonLayer.from_geopandas(
    hull_gdf,
    get_fill_color=[255, 0, 0, 63],
    stroked=True,
    get_line_width=200,
    get_line_color=[0, 0, 0, 150],
)

tile_layer = BitmapTileLayer(
    # data="https://tile.openstreetmap.org/{z}/{x}/{y}.png",
    data=tile_url,
    tile_size=256,
    max_requests=-1,
    min_zoom=0,
    max_zoom=19,
)

m = Map(
    layers=[
        gridcells_layer,
        tile_layer,
        hull_layer
    ],
    controls=[]
)

checkbox_hull = widgets.Checkbox(
    value=True,
    description='Hull',
)
checkbox_tiles = widgets.Checkbox(
    value=True,
    description='Tiles',
)
checkbox_gridcells = widgets.Checkbox(
    value=True,
    description='Gridcells',
)

# Link checkboxes to layer visibility
widgets.link((checkbox_hull, 'value'), (hull_layer, 'visible'))
widgets.link((checkbox_tiles, 'value'), (tile_layer, 'visible'))
widgets.link((checkbox_gridcells, 'value'), (gridcells_layer, 'visible'))

controls = widgets.HBox([checkbox_tiles, checkbox_gridcells, checkbox_hull])
display(controls, m)

In [13]:
aoo_area_km2 = hull.area().multiply(1e-6).getInfo()
print(f'The area of the hull is {aoo_area_km2:.0f} km²')

The area of the hull is 50337 km²


## Verify that the step-by-step results are consistent

In [14]:
# Direct call to `make_eoo()`
aoo_area_km2_direct_call = area_km2(make_eoo(class_img)).getInfo()
print(f'EOO area: {aoo_area_km2_direct_call:.0f} km²')

EOO area: 50337 km²


In [15]:
assert aoo_area_km2 == aoo_area_km2_direct_call

# Area of Occupancy (AOO) (subcriterion B2)

The protocol for this adjustment includes the following steps:

- Intersect AOO grid with the ecosystem’s distribution map.
- Calculate extent of the ecosystem type in each grid cell (‘area’) and sum these areas to obtain the total ecosystem area (‘total area’).
- Arrange grid cells in ascending order based on their area (smaller first). - Calculate accumulated sum of area per cell (‘cumulative area’).
- Calculate ‘cumulative proportion’ by dividing ‘cumulative area’ by ‘total area’ (cumulative proportion takes values between 0 and 1).
- Calculate AOO by counting the number of cells with a ‘cumulative proportion’ greater than 0.01 (i.e. exclude cells that in combination account for up to 1% of the total mapped extent of the ecosystem type).

In [16]:
# aoo_grid_proj = get_aoo_grid_projection()

# fcov_temp = class_img.unmask().reduceResolution(
#     reducer=ee.Reducer.mean(),
#     maxPixels=65536
#   ).reproject(aoo_grid_proj)

# # Mask out zero values.
# fractional_coverage = fcov_temp.mask(fcov_temp.gt(0))

In [24]:
aoo_grid_proj = get_aoo_grid_projection()

# Test reducing directly to a feature collection, without first reprojecting
fractional_coverage_fc = class_img.unmask().reduceRegions(
  collection=class_img.geometry().coveringGrid(aoo_grid_proj),
  reducer=ee.Reducer.mean(),
).filter(ee.Filter.gt('mean', 0))

fractional_coverage_gdf = ee.data.computeFeatures({
    "expression": fractional_coverage_fc,
    "fileFormat": "GEOPANDAS_GEODATAFRAME",
})

fractional_coverage_gdf.rename(columns={"mean": "coverage"}, inplace=True)

print(fractional_coverage_gdf.sort_values(by="coverage")[1:4])

                                              geometry  coverage
125  POLYGON ((98.09603 11.93102, 98.18586 11.93102...  0.000079
111  POLYGON ((98.09603 11.74632, 98.18586 11.74632...  0.000079
74   POLYGON ((98.54519 11.1008, 98.63502 11.1008, ...  0.000079


In [18]:
len(fractional_coverage_gdf)

206

In [19]:
# aoo_grid_tile_url = fractional_coverage.getMapId(
#         vis_params={
#             'palette': ['green'],
#             'opacity': 0.5
#         }
#     )['tile_fetcher'].url_format

# aoo_grid_tile_layer = BitmapTileLayer(
#     data=aoo_grid_tile_url,
#     tile_size=256,
#     max_requests=-1,
#     min_zoom=0,
#     max_zoom=19,
# )

In [25]:
fractional_coverage_gdf_layer = PolygonLayer.from_geopandas(
    fractional_coverage_gdf.sort_values(by="coverage")[1:4],
    get_fill_color=[255, 0, 0, 63],
)

m = Map(
    layers=[
        tile_layer,
        fractional_coverage_gdf_layer
        # aoo_grid_tile_layer,
    ],
    controls=[]
)

controls = widgets.HBox([])
display(controls, m)

/Users/tylere/Documents/GitHub/VorGeo/rle-assessment-report/.pixi/envs/default/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:40: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


HBox()

Trying to convert to a FeatureCollection directly (i.e. `fractionalCoverage.reduceRegions()`) results in an error:
`EEException: Reprojection output too large (14558x23872 pixels)`

To avoid this, we export the intermediate image as an asset:

```
        export_fractional_coverage_on_aoo_grid(
            class_img=class_img,
            asset_id=asset_id,
            export_description=f"AOO_grid_image_for_{ecosystem['code']}",
            create_folder=False,
        )
```

In [ ]:
# # Construct the exported asset id
# exported_asset_id = f"{gee_project_path}/{class_info['code'].replace('.', '_')}/a00_grid"
# print(f'{exported_asset_id = }')

In [ ]:
# # Load the exported asset
# fractional_coverage_exported = ee.Image(exported_asset_id)

In [ ]:
# # Verify that the intermediate asset and exported asset are identical

# # Check if the assets are identical
# diff = fractional_coverage.subtract(fractional_coverage_exported)

# diff_tile_url = fractional_coverage.getMapId(
#         vis_params={
#             'palette': ['red', 'white', 'blue'],
#             'opacity': 0.5,
#             'min': -1,
#             'max': 1
#         }
#     )['tile_fetcher'].url_format

# diff_tile_layer = BitmapTileLayer(
#     data=diff_tile_url,
#     tile_size=256,
#     max_requests=-1,
#     min_zoom=0,
#     max_zoom=19,
# )


In [ ]:
# m_diff = Map(
#     layers=[
#         tile_layer,
#         diff_tile_layer,
#     ],
#     controls=[]
# )
# controls2 = widgets.HBox([])
# display(controls2, m_diff)

In [ ]:
# # Determine the fractional coverage in each AOO grid cell
# fractional_coverage_fc = fractional_coverage_exported.reduceRegions(
#   collection=fractional_coverage_exported.geometry().coveringGrid(aoo_grid_proj),
#   reducer=ee.Reducer.mean(),
# ).filter(ee.Filter.gt('mean', 0)).select('mean', 'coverage')

In [ ]:
# fractional_coverage_fc_count = fractional_coverage_fc.size().getInfo()
# print(f'{fractional_coverage_fc_count = }')

In [ ]:
# Display the FeatureCollection using a MapId & BitmapTileLayer
# aoo_grid_cells_tile_url = aoo_grid_cells.getMapId(
#         vis_params={
#             'color': 'blue'
#         }
#     )['tile_fetcher'].url_format

# aoo_grid_cells_tile_layer = BitmapTileLayer(
#     data=aoo_grid_cells_tile_url,
#     tile_size=256,
#     max_requests=-1,
#     min_zoom=0,
#     max_zoom=19,
#     opacity=0.1
# )

In [ ]:
# # Confirm that the area of the grid cells is correct
# total_grid_cell_area = aoo_grid_cells.geometry().area(1, aoo_grid_proj).getInfo()
# total_grid_cell_area

- Calculate extent of the ecosystem type in each grid cell (‘area’) and sum these areas to obtain the total ecosystem area (‘total area’).

In [ ]:
# # Convert to a GeoPandas GeoDataFrame
# fractional_coverage_gdf = ee.data.computeFeatures({
#     "expression": fractional_coverage_fc,
#     "fileFormat": "GEOPANDAS_GEODATAFRAME",
# })

In [ ]:
# # !!!!! Temporary fix until intermediate asset is exported.
# fractional_coverage_gdf.rename(columns={"mean": "coverage"}, inplace=True)

- Arrange grid cells in ascending order based on their area (smaller first). - Calculate accumulated sum of area per cell (‘cumulative area’).

In [ ]:
fractional_coverage_gdf.sort_values(by="coverage")[1:4]

In [ ]:
aoo_gdf_layer = PolygonLayer.from_geopandas(
    fractional_coverage_gdf.sort_values(by="coverage")[1:4],
    get_fill_color=[255, 0, 0, 63],
)

m2 = Map(
    layers=[
        tile_layer,
        # aoo_grid_tile_layer,
        # aoo_grid_cells_tile_layer,
        aoo_gdf_layer
    ],
    controls=[]
)

controls2 = widgets.HBox([])
display(controls2, m2)

- Calculate ‘cumulative proportion’ by dividing ‘cumulative area’ by ‘total area’ (cumulative proportion takes values between 0 and 1).

- Calculate AOO by counting the number of cells with a ‘cumulative proportion’ greater than 0.01 (i.e. exclude cells that in combination account for up to 1% of the total mapped extent of the ecosystem type).

## Verify that the step-by-step results are consistent

In [ ]:
# # Direct call to `make_aoo()`
# aoo_grid_cell_count_direct_call = area_km2(make_aoo(class_img)).getInfo()
# print(f'AOO: {aoo_grid_cell_count_direct_call} grid cells')

In [ ]:
# assert aoo_grid_cell_count == aoo_grid_cell_count_direct_call

# Criterion B Summary

In [ ]:
print(f'As shown in the preceding sections, '
      f'AOO and EOO were measured as '
      f'{aoo_grid_cell_count} 10 x 10 km grid cells '
      f'and {aoo_area_km2:.0f} km², respectively.')